Open up data and seperate for different types, no time series because its developed further in the file

In [2]:
import os
import re
import pandas as pd
import numpy as np

# Helper function
def get_label(filename):
    match = re.match(r'^(als|hunt|control|park)', filename.lower())
    if match:
        return match.group(1)
    return 'unknown'

# Function to decode header file
def decode_header(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    header_dict = {}
    lines = [line.strip() for line in lines if line.strip() and not line.startswith('#')]
    if not lines:
        return header_dict

    first_line = lines[0]
    fields = first_line.split()
    if len(fields) >= 5:
        header_dict['data_file'] = fields[0]
        
        header_dict['channels'] = int(fields[1])
        
        header_dict['sampling_rate'] = float(fields[2])
        
        header_dict['calibration_factor'] = float(fields[3])
        
        header_dict['gain'] = int(fields[4])

        for i, val in enumerate(fields[5:], start=5):
            header_dict[f'field_{i}'] = val

    # Process subsequent lines in "key: value" format.
    for line in lines[1:]:
        if ':' in line:
            key, val = line.split(':', 1)
            header_dict[key.strip()] = val.strip()

    return header_dict

# Function to decode binary signal files (.let and .rit)
def decode_signal(file_path, dtype=np.int16):
    with open(file_path, 'rb') as f:
        data = f.read()
    # Convert the binary data into a NumPy array
    signal = np.frombuffer(data, dtype=dtype)
    df = pd.DataFrame({
        'sample_index': np.arange(len(signal)),
        'signal_value': signal
    })
    return df

folder_path = 'gait_data'

headers_list = []
left_signals = []
right_signals = []

# Process each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if not os.path.isfile(file_path):
        continue

    label = get_label(filename)
    
    if filename.endswith('.hea'):
        header_info = decode_header(file_path)
        header_info['filename'] = filename
        header_info['label'] = label
        headers_list.append(header_info)
    
    elif filename.endswith('.let'):
        df_left = decode_signal(file_path)
        df_left['filename'] = filename
        df_left['label'] = label
        df_left['side'] = 'left'
        left_signals.append(df_left)
    
    elif filename.endswith('.rit'):
        df_right = decode_signal(file_path)
        df_right['filename'] = filename
        df_right['label'] = label
        df_right['side'] = 'right'
        right_signals.append(df_right)


# Create final DataFrames for each file type
df_headers = pd.DataFrame(headers_list)
df_left_signals = pd.concat(left_signals, ignore_index=True) if left_signals else pd.DataFrame()
df_right_signals = pd.concat(right_signals, ignore_index=True) if right_signals else pd.DataFrame()


print("Decoded Header DataFrame:")
print(df_headers.head(), "\n")

print("Decoded Left Foot Signal DataFrame:")
print(df_left_signals.head(), "\n")

print("Decoded Right Foot Signal DataFrame:")
print(df_right_signals.head(), "\n")

Decoded Header DataFrame:
    filename label
0   als1.hea   als
1  als10.hea   als
2  als11.hea   als
3  als12.hea   als
4  als13.hea   als 

Decoded Left Foot Signal DataFrame:
   sample_index  signal_value  filename label  side
0             0         -8878  als1.let   als  left
1             1         21330  als1.let   als  left
2             2         21469  als1.let   als  left
3             3         -8877  als1.let   als  left
4             4         21331  als1.let   als  left 

Decoded Right Foot Signal DataFrame:
   sample_index  signal_value  filename label   side
0             0         -6144  als1.rit   als  right
1             1        -10794  als1.rit   als  right
2             2        -10514  als1.rit   als  right
3             3         -4394  als1.rit   als  right
4             4        -10537  als1.rit   als  right 



Develope timeseries files, and how they interact

In [3]:
import os
import pandas as pd

def extract_double_support(file_path):
    """
    Reads a .ts file and extracts the Double Support Time column.
    
    Assumes the .ts file is whitespace-delimited and that:
      - The file does not have a header row.
      - The 12th column (index 11) represents the Double Support Time.
    
    Parameters:
        file_path (str): Path to the .ts file.
        
    Returns:
        pd.DataFrame: A DataFrame with two columns:
            - 'Double Support Time': The double support time for each stride.
            - 'filename': The name of the file.
    """
    # Read the .ts file
    df = pd.read_csv(file_path, sep=r'\s+', header=None, engine='python')
    df.dropna(how='all', inplace=True)
    
    # Extract the double support time column
    double_support_time = df.iloc[:, 11]
    
    # Get the filename
    file_name = os.path.basename(file_path)

    label_name = re.match(r'^(als|hunt|control|park)', file_name.lower())
    
    # Create a new DataFrame with the desired two columns.
    result_df = pd.DataFrame({
        'Double Support Time': double_support_time,
        'filename': file_name,
        'label': label_name.group(1)
    })
    
    return result_df

# Dataframes of our variables
file_als = 'gait_data/als1.ts'
df_double_support_als = extract_double_support(file_als)
print(len(df_double_support_als))

file_hunt = 'gait_data/hunt1.ts'
df_double_support_hunt = extract_double_support(file_hunt)
print(len(df_double_support_hunt))

file_control = 'gait_data/control1.ts'
df_double_support_control = extract_double_support(file_control)
print(len(df_double_support_control))

file_park = 'gait_data/park1.ts'
df_double_support_park = extract_double_support(file_park)
print(len(df_double_support_park))

194
310
259
245


Combine all the singular time series df into one big combined timeseries df

In [4]:
import pandas as pd

# Determine the smallest number of rows among the four DataFrames
min_length = min(
    len(df_double_support_als),
    len(df_double_support_hunt),
    len(df_double_support_control),
    len(df_double_support_park)
)

# Trim each DataFrame to the minimum number of rows and reset their index
df_als_trim = df_double_support_als.iloc[:min_length].reset_index(drop=True)
df_hunt_trim = df_double_support_hunt.iloc[:min_length].reset_index(drop=True)
df_control_trim = df_double_support_control.iloc[:min_length].reset_index(drop=True)
df_park_trim = df_double_support_park.iloc[:min_length].reset_index(drop=True)

# Create a combined DataFrame with one column per variable.
df_combined = pd.DataFrame({
    'als': df_als_trim['Double Support Time'],
    'hunt': df_hunt_trim['Double Support Time'],
    'control': df_control_trim['Double Support Time'],
    'park': df_park_trim['Double Support Time']
})

# Display the combined DataFrame
df_combined


,als,hunt,control,park
0,0.4633,0.1900,0.3200,0.4333
1,0.4400,0.1800,0.3067,0.3767
2,0.4267,0.1933,0.2933,0.3800
3,0.5467,0.1867,0.3233,0.4133
4,0.5333,0.1933,0.2767,0.3800
...,...,...,...,...
189,0.5100,0.1967,0.3600,0.3500
190,0.5333,0.1833,0.3300,0.3000
191,0.5300,0.1967,0.3533,0.3067
192,0.5400,0.2067,0.3933,0.3367


In [5]:
import plotly.express as px
import pandas as pd
import numpy as np

# Combine the individual DataFrames for each group into one DataFrame.
df_all = pd.concat([
    df_double_support_als,
    df_double_support_hunt,
    df_double_support_control,
    df_double_support_park
], ignore_index=True)

# Filter out rows with Double Support Time above 1.
df_all = df_all[df_all['Double Support Time'] <= 1]

# Map each group to a numeric value for the x-axis.
group_mapping = {'als': 0, 'hunt': 1, 'control': 2, 'park': 3}
df_all['group_numeric'] = df_all['label'].map(group_mapping)

# Add a small horizontal jitter to avoid overlapping dots.
df_all['jitter'] = df_all['group_numeric'] + np.random.uniform(-0.1, 0.1, size=len(df_all))

# Create the scatter plot using Plotly Express.
fig = px.scatter(
    df_all,
    x='jitter',
    y='Double Support Time',
    color='label',
    opacity=0.7,
    title="Double Support Time per Group (Filtered: Values <= 1)",
    labels={'jitter': 'Group', 'Double Support Time': 'Double Support Time (s)'}
)

# Adjust the x-axis ticks to show group names.
fig.update_xaxes(
    tickmode='array',
    tickvals=[0, 1, 2, 3],
    ticktext=['als', 'hunt', 'control', 'park']
)

fig.show()


In [6]:
# convert to json
df_json = df_all.to_json(orient="records")
with open("data.json", "w") as f:
    f.write(df_json)

In [7]:
#### Get left and right strides for each group
import os
import pandas as pd

def extract_strides(file_path):
    """
    Reads a .ts file and extracts the Double Support Time column.
    
    Assumes the .ts file is whitespace-delimited and that:
      - The file does not have a header row.
      - Columns 1, 2, 3, 4 are for left stride, right stride, left swing, right swing respectively.
    
    Parameters:
        file_path (str): Path to the .ts file.
        
    Returns:
        pd.DataFrame: A DataFrame with two columns:
            - 'Double Support Time': The double support time for each stride.
            - 'filename': The name of the file.
    """
    # Read the .ts file
    df = pd.read_csv(file_path, sep=r'\s+', header=None, engine='python')
    df.dropna(how='all', inplace=True)
    
    # Extract the double support time column
    left_stride = df.iloc[:, 1]
    right_stride = df.iloc[:, 2]
    left_swing = df.iloc[:, 3]
    right_swing = df.iloc[:, 4]

    # Get the filename
    file_name = os.path.basename(file_path)

    label_name = re.match(r'^(als|hunt|control|park)', file_name.lower())
    
    # Create a new DataFrame with the desired two columns.
    result_df = pd.DataFrame({
        'left_stride': left_stride,
        'right_stride': right_stride,
        'left_swing': left_swing,
        'right_swing': right_swing,
        'filename': file_name,
        'label': label_name.group(1)
    })
    
    return result_df

# Dataframes of our variables
file_als = 'gait_data/als1.ts'
df_strides_als = extract_strides(file_als)
print(len(df_double_support_als))

file_hunt = 'gait_data/hunt1.ts'
df_strides_hunt = extract_strides(file_hunt)
print(len(df_double_support_hunt))

file_control = 'gait_data/control1.ts'
df_strides_control = extract_strides(file_control)
print(len(df_double_support_control))

file_park = 'gait_data/park1.ts'
df_strides_park = extract_strides(file_park)
print(len(df_double_support_park))

194
310
259
245


In [8]:
df_strides_als.head()

,left_stride,right_stride,left_swing,right_swing,filename,label
0,1.2833,1.3533,0.4067,0.4133,als1.ts,als
1,1.3233,1.2667,0.4833,0.4000,als1.ts,als
2,1.3033,1.3600,0.4500,0.4267,als1.ts,als
3,1.4167,1.2833,0.5033,0.3667,als1.ts,als
4,1.2367,1.4067,0.3467,0.3567,als1.ts,als


In [9]:
# Determine the smallest number of rows among the four DataFrames
min_length = min(
    len(df_strides_als),
    len(df_strides_hunt),
    len(df_strides_control),
    len(df_strides_park)
)

# Trim each DataFrame to the minimum number of rows and reset their index
df_als_trim = df_strides_als.iloc[:min_length].reset_index(drop=True)
df_hunt_trim = df_strides_hunt.iloc[:min_length].reset_index(drop=True)
df_control_trim = df_strides_control.iloc[:min_length].reset_index(drop=True)
df_park_trim = df_strides_park.iloc[:min_length].reset_index(drop=True)



In [10]:
def save_to_json(df, filename):
    try:
        df_json = df.to_json(orient="records")
        with open(filename, "w") as f:
            f.write(df_json)
        print(f"File {filename} created successfully.")
    except Exception as e:
        print(f"Failed to create file {filename}: {e}")

# Save DataFrames to JSON files
save_to_json(df_strides_als, "als1.json")
save_to_json(df_strides_hunt, "hunt1.json")
save_to_json(df_strides_control, "control1.json")
save_to_json(df_strides_park, "park1.json")

File als1.json created successfully.
File hunt1.json created successfully.
File control1.json created successfully.
File park1.json created successfully.


So i was thinking as one interactive feature can be that when you click on a certain disease it will focus on it enlarge it and show on it the average for that group compared to average for control and average over all the variables

The next markdown is the summary/overview part of our prototype

We have decided to remain the same group for the final project after Project 3. Since we have already developed numerous ideas regarding the previous dataset, KoreanDB, we wanted to explore something new and more challenging. This led us to research various topics within a larger resource called PhysioNet. We were particularly drawn to this dataset because, at first glance, it presented many opportunities to develop new visualization skills.

So far, we have completed several key tasks. Firstly, we brainstormed ideas that interest us and could effectively convey the essence of our final project—telling a compelling story about how and why people with certain diseases struggle with gait abnormalities. Secondly, since this is a challenging dataset, we familiarized ourselves with the available tools, explored the different data types in the files, and gained a general understanding of how the variables interact. This step was moderately difficult because the dataset was formatted in a way that differed from the usual CSV files we are accustomed to, requiring us to revisit and refine our file manipulation skills.

We still need to finalize the exact direction of our project, which presents several challenges. One of the biggest difficulties is determining the most effective way to convey the physical disparities between different disease groups. An integral part of this challenge is incorporating animations on our website—whether through smooth transitions or fully interactive motion graphics. We have never created animations before, so we are unsure whether they will enhance or reduce the clarity of our visualization. While animations can improve engagement, they must serve the story—if they fail to do so, they become distracting rather than useful.

Last but not least, storytelling itself will be our biggest challenge. In past projects, we focused heavily on developing technical skills. However, for this project, we must go beyond the technical execution and focus on crafting a compelling narrative—because a truly great visualization is built on a strong story. Although we have put much thought into this aspect, we recognize that storytelling is what will make our visualization impactful, and it requires just as much effort as the technical implementation.

GOAL: Compute average walking speed using subject_description.txt

In [11]:
import pandas as pd
import numpy as np

def compute_avg_gait_speed(file_path, disease_name):
    try:
        df = pd.read_csv(file_path, sep='\t', engine='python')
    except:
        df = pd.read_csv(file_path, sep=r'\s+', engine='python')

    df.columns = ['Subject', 'Group', 'Age', 'Height', 'Weight', 'Gender', 'GaitSpeed', 'Severity']

    df['GaitSpeed'] = pd.to_numeric(df['GaitSpeed'], errors='coerce')

    if disease_name.lower() == "als":
        df_filtered = df[df['Subject'].str.contains("als", case=False, na=False)]
    else:
        df_filtered = df[df['Group'].str.contains(disease_name, case=False, na=False)]

    return df_filtered['GaitSpeed'].mean()

file_path = "gait_data/subject-description.txt"

avg_gait_speed_control = compute_avg_gait_speed(file_path, "control")
avg_gait_speed_hunt = compute_avg_gait_speed(file_path, "hunt")
avg_gait_speed_park = compute_avg_gait_speed(file_path, "park")
avg_gait_speed_als = compute_avg_gait_speed(file_path, "als")

print(f"Average Gait Speed for Control: {avg_gait_speed_control:.2f} m/s")
print(f"Average Gait Speed for Huntington's: {avg_gait_speed_hunt:.2f} m/s")
print(f"Average Gait Speed for Parkinson's: {avg_gait_speed_park:.2f} m/s")
print(f"Average Gait Speed for ALS: {avg_gait_speed_als:.2f} m/s")


Average Gait Speed for Control: 1.35 m/s
Average Gait Speed for Huntington's: 1.15 m/s
Average Gait Speed for Parkinson's: 1.00 m/s
Average Gait Speed for ALS: 1.05 m/s


GOAL: get positionality and time for each person

In [13]:
import pandas as pd
import numpy as np

# -------------------------------
# Read Gait Data
# -------------------------------
file_path = 'gait_data/park1.ts'
data = pd.read_csv(file_path, delimiter="\t", header=0)
data.columns = [
    'elapsed_time', 'left_stride', 'right_stride', 'left_swing', 'right_swing',
    'left_swing_pct', 'right_swing_pct', 'left_stance', 'right_stance',
    'left_stance_pct', 'right_stance_pct', 'double_support_sec', 'double_support_pct'
]

# -------------------------------
# Simulation Parameters
# -------------------------------
body_height = 2  # Replace with actual subject height
H_peak = 0.06 * body_height  # Peak foot clearance (6% of body height)
dt = 0.01  # Time resolution
gait_speed = 0.98  # m/s

# -------------------------------
# Parabolic Vertical Motion Function
# -------------------------------
def parabolic_height(t, T_swing, H_peak):
    return - (4 * H_peak / T_swing**2) * (t - T_swing/2)**2 + H_peak

# -------------------------------
# Full Gait Simulation
# -------------------------------
def simulate_full_gait(data):
    results = []
    current_time = 0

    # Adjusted Initial Positions
    initial_step_length = gait_speed * data['right_stride'].iloc[0]
    left_x, right_x = 0.0, initial_step_length / 2
    left_y, right_y = 0.0, 0.0

    for _, row in data.iterrows():
        # Extract parameters
        T_double = row['double_support_sec']
        T_stride_L = row['left_stride']
        T_swing_L = row['left_swing']
        T_stance_L = row['left_stance']
        
        T_stride_R = row['right_stride']
        T_swing_R = row['right_swing']
        T_stance_R = row['right_stance']

        # Phase 1: Double Support
        t_vals = np.arange(current_time, current_time + T_double, dt)
        for t in t_vals:
            results.append({'time': t, 'x_left': left_x, 'x_right': right_x, 'y_left': 0.0, 'y_right': 0.0})
        current_time += T_double

        # Phase 2: Left Foot Swing
        step_length_L = gait_speed * T_stride_L
        swing_speed_L = step_length_L / T_swing_L
        t_swing = np.arange(0, T_swing_L, dt)
        for t in t_swing:
            x = left_x + swing_speed_L * t
            y = parabolic_height(t, T_swing_L, H_peak)
            results.append({'time': current_time + t, 'x_left': x, 'x_right': right_x, 'y_left': y, 'y_right': 0.0})
        left_x += step_length_L
        current_time += T_swing_L

        # Phase 3: Left Stance
        t_vals = np.arange(current_time, current_time + T_stance_L - T_double, dt)
        for t in t_vals:
            results.append({'time': t, 'x_left': left_x, 'x_right': right_x, 'y_left': 0.0, 'y_right': 0.0})
        current_time += (T_stance_L - T_double)

        # Phase 4: Double Support
        t_vals = np.arange(current_time, current_time + T_double, dt)
        for t in t_vals:
            results.append({'time': t, 'x_left': left_x, 'x_right': right_x, 'y_left': 0.0, 'y_right': 0.0})
        current_time += T_double

        # Phase 5: Right Foot Swing
        step_length_R = gait_speed * T_stride_R
        swing_speed_R = step_length_R / T_swing_R
        t_swing = np.arange(0, T_swing_R, dt)
        for t in t_swing:
            x = right_x + swing_speed_R * t
            y = parabolic_height(t, T_swing_R, H_peak)
            results.append({'time': current_time + t, 'x_left': left_x, 'x_right': x, 'y_left': 0.0, 'y_right': y})
        right_x += step_length_R
        current_time += T_swing_R

        # Phase 6: Right Stance
        t_vals = np.arange(current_time, current_time + T_stance_R - T_double, dt)
        for t in t_vals:
            results.append({'time': t, 'x_left': left_x, 'x_right': right_x, 'y_left': 0.0, 'y_right': 0.0})
        current_time += (T_stance_R - T_double)

    return pd.DataFrame(results)


# -------------------------------
# Run Simulation
# -------------------------------
df = simulate_full_gait(data)
save_to_json(df, 'gait_coordinates/park1_coord.json')


File gait_coordinates/park1_coord.json created successfully.
